# Interpolation
In this notebook, we will talk about interpolating coordinate values from an image.

In [ ]:
def interpolate(image, coords, FOV):
    ft_image = np.fft.fftshift(np.fft.fft2(image))

    kx_1 = 2*math.pi/FOV

    kx = np.linspace(ft_image.shape[0]//-2, (ft_image.shape[0])//2 - 1, ft_image.shape[0]) * kx_1
    ky = np.linspace(ft_image.shape[0]//-2, (ft_image.shape[1])//2 - 1, ft_image.shape[1]) * kx_1
    
    interp_real = RegularGridInterpolator((kx, ky), ft_image.real, bounds_error=False, method="linear")
    interp_imag = RegularGridInterpolator((kx, ky), ft_image.imag, bounds_error=False, method="linear")

    real = interp_real(coords)
    imag = interp_imag(coords)
    return real + imag * 1j

image is a 80x80 pixel image that represents our reconstructed image
coords is a list of u,v coordinates that we obtained from our data
FOV is the Field of view from the telescopes. For the EHT data, our FOV is 100 micro ascs.

First we take the image and get it's fourier transform. We do this since our data is in the Fourier Domain and it allows us for a more direct comparison.

Next we want to interpolate the complex value at each coordinate. RegularGridInterpolator however cannot interpolate complex numbers so we interpolate the real and imaginary parts separately and then combined them for the final result.
Here we use a linear method for interpolation due to it's local interpolation scheme. The other schemes that RegularGridInterpolator offers uses a C^2-smooth split which is a non-local scheme. Since we wanted to optimize the computation time, we chose the linear method.

Image in x and y gets transformed into Fourier domain in kx and ky. The formula to transform it is: 

When we transform we need to also translate one unit from the image to one unit of the fourier image.
Thats why we multiply kx and ky by kx_1. 


# Check kx_1 with CK and Misha



The more grid points we have, the bigger the image. Thus the size of our Fourier Domain is the Number of pixels of the image by kx_1
If our FOV is very small, then the width of the fourier grid becomes large.